In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import StratifiedKFold, cross_val_score
from xgboost import XGBRegressor  
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.metrics import r2_score
import optuna
from xgboost import XGBRegressor

In [3]:
df= pd.read_csv(r"D:\Intellipaat\kegal datasets\april -s4e4\train.csv")
df.drop(columns=['id'],inplace = True)
df

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9
...,...,...,...,...,...,...,...,...,...
90610,M,0.335,0.235,0.075,0.1585,0.0685,0.0370,0.0450,6
90611,M,0.555,0.425,0.150,0.8790,0.3865,0.1815,0.2400,9
90612,I,0.435,0.330,0.095,0.3215,0.1510,0.0785,0.0815,6
90613,I,0.345,0.270,0.075,0.2000,0.0980,0.0490,0.0700,6


In [4]:
path_to_input_folder = "D:\\Intellipaat\\kegal datasets\\april -s4e4\\abalone\\"
column_names = ['Sex','Length','Diameter','Height','Whole weight','Whole weight.1','Whole weight.2','Shell weight','Rings']
org_df = pd.read_csv(path_to_input_folder + "abalone.data", names=column_names)

In [5]:
data = pd.concat([df,org_df])
data

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,F,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,F,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,I,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,M,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,I,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9
...,...,...,...,...,...,...,...,...,...
4172,F,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,M,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,M,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,F,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [6]:
data.shape

(94792, 9)

In [7]:
encoder = LabelEncoder()

In [8]:
data['Sex'] = encoder.fit_transform(data['Sex'].values.reshape(-1,1))

C:\Users\mahes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
data

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,0,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,1,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,2,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,1,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9
...,...,...,...,...,...,...,...,...,...
4172,0,0.565,0.450,0.165,0.8870,0.3700,0.2390,0.2490,11
4173,2,0.590,0.440,0.135,0.9660,0.4390,0.2145,0.2605,10
4174,2,0.600,0.475,0.205,1.1760,0.5255,0.2875,0.3080,9
4175,0,0.625,0.485,0.150,1.0945,0.5310,0.2610,0.2960,10


In [10]:
X = data.drop(columns=['Rings']) 
Y = data['Rings']

In [11]:
data.head()

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight,Rings
0,0,0.550,0.430,0.150,0.7715,0.3285,0.1465,0.2400,11
1,0,0.630,0.490,0.145,1.1300,0.4580,0.2765,0.3200,11
2,1,0.160,0.110,0.025,0.0210,0.0055,0.0030,0.0050,6
3,2,0.595,0.475,0.150,0.9145,0.3755,0.2055,0.2500,10
4,1,0.555,0.425,0.130,0.7820,0.3695,0.1600,0.1975,9


In [12]:
import optuna
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor  # Import CatBoostRegressor

# Define the objective function for optimization
def objective_catboost(trial):
    cvo = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    params = dict(
        iterations=trial.suggest_int('iterations', 100, 500),  # Change to 'iterations'
        depth=trial.suggest_int('depth', 2, 16),  # Change to 'depth'
        learning_rate=trial.suggest_float('learning_rate', 0.001, 0.3),
        l2_leaf_reg=trial.suggest_float('l2_leaf_reg', 0.1, 10),
        random_strength=trial.suggest_float('random_strength', 0.1, 10),
        bagging_temperature=trial.suggest_float('bagging_temperature', 0.0, 1.0),
        border_count=trial.suggest_int('border_count', 1, 255)
    )

    catboost = CatBoostRegressor(random_state=0, **params)  # Use CatBoostRegressor

    pipe = Pipeline(
        steps=[
            ('scaler', StandardScaler()),
            ('model', catboost)  # Use CatBoostRegressor in the pipeline
        ]
    )

    score = -np.mean(cross_val_score(pipe, X, Y, scoring='neg_mean_squared_error', cv=cvo))  # Use neg_mean_squared_error for regression
    return score

# Create an Optuna study
study_catboost = optuna.create_study(direction='minimize')  # Use 'minimize' for regression problems

# Optimize the objective function
study_catboost.optimize(objective_catboost, n_trials=20, n_jobs=-1, show_progress_bar=True)

# Get the best parameters and score
best_params = study_catboost.best_params
best_score = study_catboost.best_value

print("Best Parameters:", best_params)
print("Best Score:", best_score)

# Train CatBoost regressor with the best parameters
best_catboost = CatBoostRegressor(random_state=0, **best_params)  # Use CatBoostRegressor

best_pipe = Pipeline(
    steps=[
        ('scaler', StandardScaler()),
        ('model', best_catboost)
    ]
)

# Fit the pipeline to the entire dataset
best_pipe.fit(X, Y)

# Make predictions
predictions = best_pipe.predict(X)

# Now you have predictions using the optimized CatBoost regressor


[I 2024-04-23 17:08:21,203] A new study created in memory with name: no-name-1621fb47-6f2c-4cc8-aea6-1c42abee148f


  0%|          | 0/20 [00:00<?, ?it/s]

0:	learn: 2.9144656	total: 20ms	remaining: 6.37s
0:	learn: 2.9327568	total: 13.5ms	remaining: 5.39s
1:	learn: 2.7592857	total: 25.3ms	remaining: 5.05s
0:	learn: 3.1445525	total: 15.8ms	remaining: 2.08s
1:	learn: 2.7259420	total: 89.8ms	remaining: 14.3s
1:	learn: 3.1158573	total: 28.9ms	remaining: 1.89s
2:	learn: 2.5985182	total: 108ms	remaining: 11.5s
0:	learn: 3.1511751	total: 30.5ms	remaining: 12.7s
3:	learn: 2.5000647	total: 147ms	remaining: 11.6s
2:	learn: 2.6259002	total: 93.6ms	remaining: 12.4s
0:	learn: 2.9197625	total: 86.7ms	remaining: 30.9s
2:	learn: 3.0829166	total: 95.4ms	remaining: 4.13s
3:	learn: 2.5257687	total: 111ms	remaining: 11s
0:	learn: 3.0258948	total: 317ms	remaining: 1m 39s
4:	learn: 2.4437357	total: 173ms	remaining: 10.9s
3:	learn: 3.0508864	total: 112ms	remaining: 3.61s
4:	learn: 2.4540155	total: 127ms	remaining: 10.1s
0:	learn: 3.1435997	total: 33.4ms	remaining: 7.53s
0:	learn: 2.8092614	total: 25.4ms	remaining: 3.28s
5:	learn: 2.3729073	total: 200ms	remainin

In [13]:
best_catboost

In [14]:
best_params

{'iterations': 385,
 'depth': 6,
 'learning_rate': 0.19313687753138933,
 'l2_leaf_reg': 3.792383354102723,
 'random_strength': 0.303536849801812,
 'bagging_temperature': 0.4628325289614952,
 'border_count': 214}

In [15]:
best_score

3.4022022084296437

In [16]:
best_pipe

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x0000026C23E9FED0>)])

In [17]:
predictions

array([10.35954977, 10.24672742,  4.03633884, ..., 11.04923555,
        8.62382486, 11.36214837])

In [18]:
study_catboost

In [19]:
test_data= pd.read_csv(r"D:\Intellipaat\kegal datasets\april -s4e4\test.csv")
test_data

,id,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight
0,90615,M,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005
1,90616,M,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750
2,90617,M,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405
3,90618,M,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350
4,90619,I,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050
...,...,...,...,...,...,...,...,...,...
60406,151021,I,0.345,0.260,0.085,0.1775,0.0735,0.0265,0.0500
60407,151022,F,0.525,0.410,0.145,0.8445,0.3885,0.1670,0.2050
60408,151023,I,0.590,0.440,0.155,1.1220,0.3930,0.2000,0.2650
60409,151024,F,0.660,0.525,0.190,1.4935,0.5885,0.3575,0.4350


In [20]:
test = test_data.drop(columns=['id'])
test

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight
0,M,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005
1,M,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750
2,M,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405
3,M,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350
4,I,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050
...,...,...,...,...,...,...,...,...
60406,I,0.345,0.260,0.085,0.1775,0.0735,0.0265,0.0500
60407,F,0.525,0.410,0.145,0.8445,0.3885,0.1670,0.2050
60408,I,0.590,0.440,0.155,1.1220,0.3930,0.2000,0.2650
60409,F,0.660,0.525,0.190,1.4935,0.5885,0.3575,0.4350


In [21]:
test['Sex'] = encoder.fit_transform(test_data['Sex'].values.reshape(-1,1))

C:\Users\mahes\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [22]:
test

,Sex,Length,Diameter,Height,Whole weight,Whole weight.1,Whole weight.2,Shell weight
0,2,0.645,0.475,0.155,1.2380,0.6185,0.3125,0.3005
1,2,0.580,0.460,0.160,0.9830,0.4785,0.2195,0.2750
2,2,0.560,0.420,0.140,0.8395,0.3525,0.1845,0.2405
3,2,0.570,0.490,0.145,0.8740,0.3525,0.1865,0.2350
4,1,0.415,0.325,0.110,0.3580,0.1575,0.0670,0.1050
...,...,...,...,...,...,...,...,...
60406,1,0.345,0.260,0.085,0.1775,0.0735,0.0265,0.0500
60407,0,0.525,0.410,0.145,0.8445,0.3885,0.1670,0.2050
60408,1,0.590,0.440,0.155,1.1220,0.3930,0.2000,0.2650
60409,0,0.660,0.525,0.190,1.4935,0.5885,0.3575,0.4350


In [24]:
predictions = best_pipe.predict(test)
predictions

array([ 9.4249702 ,  9.70706395, 10.10651229, ..., 13.52706414,
       13.95217716,  8.22448886])

In [25]:
# file = np.round(predictions).astype(int)
# file

array([10, 10, 10, ..., 12, 13,  9])

In [25]:
submission_df = pd.DataFrame({'id': test_data['id'], 'Rings' : predictions})
submission_df

,id,Rings
0,90615,9.424970
1,90616,9.707064
2,90617,10.106512
3,90618,10.748459
4,90619,7.488462
...,...,...
60406,151021,6.507132
60407,151022,9.629032
60408,151023,13.527064
60409,151024,13.952177


In [26]:
submission_df.to_csv('submission_parameters_1.csv', index=False)